### Menu Genie Space

This stage deploys a Databricks Genie space connected to the processed menu document
Delta tables. Users can ask natural language questions about menu items, nutritional
information, allergens, and brand comparisons.

In [ ]:
%pip install --upgrade databricks-sdk

In [ ]:
dbutils.library.restartPython()

In [ ]:
CATALOG = dbutils.widgets.get("CATALOG")

##### Create a SQL warehouse for the Genie space

In [ ]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

import sys
sys.path.append('../utils')
from uc_state import add

WAREHOUSE_NAME = f"{CATALOG}-menu-warehouse"

existing_wh = [wh for wh in w.warehouses.list() if wh.name == WAREHOUSE_NAME]
if existing_wh:
    warehouse = existing_wh[0]
    print(f"Using existing warehouse: {warehouse.id}")
else:
    warehouse = w.warehouses.create(
        name=WAREHOUSE_NAME,
        cluster_size="2X-Small",
        max_num_clusters=1,
        min_num_clusters=1,
        enable_serverless_compute=True,
    ).result()
    print(f"\u2705 Created warehouse: {warehouse.id}")

add(CATALOG, "warehouses", warehouse)

##### Deploy the Genie space via REST API

In [ ]:
import json
import hashlib

GENIE_SPACE_TITLE = f"Caspers Menu and Safety Intelligence ({CATALOG})"

GENIE_DESCRIPTION = """You are a data analyst for Caspers Kitchens, a ghost kitchen network
operating 16 restaurant brands across 4 locations (San Francisco, Silicon Valley,
Bellevue, Chicago). You have access to menu data and food safety inspection records
organized in a bronze/silver/gold medallion architecture.

=== DATA MODEL ===

SILVER TABLES (cleaned, enriched - use for detailed drill-downs):

silver_menu_items: Every menu item with enriched fields.
  Key columns: brand_name, cuisine, item_name, description, category, price,
  calories, protein_g, fat_g, carbs_g, allergens (array), allergen_count,
  is_allergen_free (boolean), price_tier (budget/standard/premium),
  calorie_category (light/moderate/hearty), protein_pct (percentage of calories
  from protein), is_high_protein (>25% calories from protein), is_low_calorie (<400 cal).

silver_inspections: Inspection records with derived compliance fields.
  Key columns: inspection_id, location_id, location_name, address, inspection_date,
  inspector_name, score (0-100), grade (A/B/C/F), violation_count, critical_count,
  major_count, minor_count, follow_up_status, passed (boolean, score >= 70),
  score_band (excellent/good/acceptable/failing), has_critical (boolean),
  severity_index (critical*3 + major*2 + minor).

silver_violations: Individual violations with urgency scoring.
  Key columns: inspection_id, location_id, location_name, inspection_date,
  code, severity (critical/major/minor), category, description, corrective_action,
  deadline_days, is_critical, is_major, is_minor (booleans),
  urgency_score (critical=10, major=5, minor=2), needs_immediate_action (deadline <= 7 days).

GOLD TABLES (business-ready views and aggregates):

menu_items: Curated item catalog (brand_name, cuisine, item_name, description, category,
  price, price_tier, allergen_count, is_allergen_free).
nutritional_info: Per-item nutrition (brand_name, item_name, category, calories, protein_g,
  fat_g, carbs_g, calorie_category, protein_pct, is_high_protein, is_low_calorie).
allergens: Per-item allergen boolean flags (contains_wheat, contains_milk, contains_egg,
  contains_soy, contains_peanut, contains_tree_nut, contains_fish, contains_shellfish,
  contains_sesame) plus allergen_count.
brand_nutrition_summary: Per-brand aggregates (total_items, avg/min/max calories,
  avg protein/fat/carbs, avg/min/max price, high_protein_items, low_calorie_items,
  allergen_free_items).
inspection_details: Inspection records (score, grade, passed, score_band, has_critical,
  severity_index, violation counts, follow_up_status).
violation_analysis: Individual violations (severity, category, corrective_action,
  deadline_days, urgency_score, needs_immediate_action).
location_compliance_summary: Per-location aggregates (total_inspections, avg/min/max score,
  total violations by severity, passed_inspections, pass_rate_pct, avg_severity_index).

=== QUERY GUIDELINES ===

- For allergen questions, use the allergens table with contains_* boolean columns.
- For nutrition comparisons, use nutritional_info or brand_nutrition_summary.
- For "healthy" questions, combine is_high_protein, is_low_calorie, and calorie_category.
- For inspection score trends, use silver_inspections and filter by location_name + inspection_date.
- For urgent safety issues, filter silver_violations WHERE needs_immediate_action = true.
- For overall location health, use location_compliance_summary (pass_rate_pct, avg_severity_index).
- price_tier: budget (<$10), standard ($10-$18), premium (>$18).
- calorie_category: light (<300), moderate (300-600), hearty (>600).
- score_band: excellent (>=90), good (80-89), acceptable (70-79), failing (<70).
- The 16 brands span cuisines: Asian, Italian, Mexican, American, Mediterranean, Indian,
  Japanese, BBQ, Seafood, Breakfast, Vegan, Southern, Greek, French, Thai, Peruvian."""

TABLE_NAMES = sorted([
    # Silver
    f"{CATALOG}.menu_documents.silver_menu_items",
    f"{CATALOG}.menu_documents.silver_inspections",
    f"{CATALOG}.menu_documents.silver_violations",
    # Gold
    f"{CATALOG}.menu_documents.menu_items",
    f"{CATALOG}.menu_documents.nutritional_info",
    f"{CATALOG}.menu_documents.allergens",
    f"{CATALOG}.menu_documents.brand_nutrition_summary",
    f"{CATALOG}.menu_documents.inspection_details",
    f"{CATALOG}.menu_documents.violation_analysis",
    f"{CATALOG}.menu_documents.location_compliance_summary",
])

serialized_space = json.dumps({
    "version": 2,
    "data_sources": {
        "tables": [{"identifier": t} for t in TABLE_NAMES],
    },
    "config": {
        "sample_questions": [
            {"id": hashlib.md5(q.encode()).hexdigest(), "question": [q]}
            for q in [
                "Which menu items are gluten-free and under $15?",
                "What is the average calorie count per brand?",
                "Which locations have critical violations that need immediate action?",
                "Compare protein content across all burgers",
                "What is the latest inspection score for Chicago?",
                "Show me all premium-tier items that are also high protein",
                "Which brand has the most allergen-free options?",
                "What is the pass rate for each location?",
                "List all violations with urgency score above 5",
                "Which cuisine type has the lowest average calories?",
            ]
        ],
    },
})

# Check for existing Genie space in uc_state, then verify it still exists
genie_space_id = None
try:
    state_df = spark.sql(f"""
        SELECT resource_data FROM {CATALOG}._internal_state.resources
        WHERE resource_type = 'genie_spaces'
        ORDER BY created_at DESC LIMIT 1
    """)
    if state_df.count() > 0:
        genie_info = json.loads(state_df.first().resource_data)
        candidate_id = genie_info.get("space_id")
        if candidate_id:
            try:
                w.genie.get_space(candidate_id)
                genie_space_id = candidate_id
                print(f"Found existing Genie space: {genie_space_id}")
            except Exception:
                print(f"Genie space {candidate_id} in state but no longer exists — will recreate")
except Exception:
    pass

if not genie_space_id:
    space = w.genie.create_space(
        warehouse_id=warehouse.id,
        serialized_space=serialized_space,
        title=GENIE_SPACE_TITLE,
        description=GENIE_DESCRIPTION,
    )
    genie_space_id = space.space_id
    print(f"\u2705 Created Genie space: {genie_space_id}")

print(f"   Title: {GENIE_SPACE_TITLE}")
print(f"   Tables: {TABLE_NAMES}")

add(CATALOG, "genie_spaces", {"space_id": genie_space_id, "title": GENIE_SPACE_TITLE})
print("\u2705 Menu Genie stage complete")